In [1]:
import pandas as pd
from epiweeks import Week
from bulletin import __file__ as __root__
from bulletin.data.casos_confirmados import CasosConfirmados

In [2]:
casos_confirmados = CasosConfirmados(force=True, hard=True)
casos_confirmados.shape()

saved checksum: 56625b0d9cabc5be8bb69f8b506ab1db62f14609e617c4b9e132b2eced042432
current checksum: 56625b0d9cabc5be8bb69f8b506ab1db62f14609e617c4b9e132b2eced042432
Tudo certo, nenhuma alteração detectada
Utilizando forcadamente com método update
Atualizando o arquivo {'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'} com o C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\Casos confirmados.xlsx...
Casos excluidos: 0
Obitos excluidos: 0
{'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'} salvo e C:\Users\est.joaosilva\covid19datascience\bulletin\tmp\casos_confirmados_checksum atualizado
{'casos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\casos.pkl', 'obitos': 'C:\\Users\\est.joaosilva\\covid19datascience\\bulletin\\tmp\\obitos.pkl'

(929097, 21564)

In [191]:
casos = casos_confirmados.get_casos()

In [192]:
sivep = pd.read_excel('output/TABELA_testes_sivep.xlsx')
notifica = pd.read_excel('output/TABELA_testes_notifica.xlsx')
notifica = notifica.set_index('SE')
sivep = sivep.set_index('SE')

In [193]:
casos['dt_diag'] = casos['dt_diag'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
casos['is'] = casos['is'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [194]:
casos['se_diag'] = casos['dt_diag'].apply(lambda x: Week.fromdate(x))
casos.loc[~casos['is'].isnull(), 'se_is'] = casos['is'].loc[~casos['is'].isnull()].apply(lambda x: Week.fromdate(x))

In [195]:
tabela_diag = casos.groupby(['se_diag'])[['nome']].count()

In [196]:
tabela_is = casos.groupby(['se_is'])[['nome']].count()
for row in tabela_is.index.values:
    if row.year < 2020:
        tabela_is = tabela_is.drop(row)

In [197]:
tabela = pd.DataFrame(sivep['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SRAG'})
notifica = pd.DataFrame(notifica['Positividade (%)']).rename(columns={'Positividade (%)': 'Positividade SG'})
tabela = tabela.merge(how = 'left', on = tabela.index, right = notifica)
tabela = tabela.rename(columns={'key_0': 'index'})

In [198]:
tabela_diag = tabela_diag.rename(columns={'nome': 'Diagnóstico'})
tabela_is = tabela_is.rename(columns={'nome':'Primeiros sintomas'})

In [199]:
index_list = pd.Series(dtype = 'int64')
for line in tabela_diag.index.values:
    index_list = index_list.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_diag = tabela_diag.set_index(index_list)
tabela_diag = tabela_diag.reset_index()
tabela = tabela.merge(how='left', on='index', right = tabela_diag)

C:\Users\est.joaosilva\Anaconda3\lib\site-packages\numpy\core\numeric.py:2378: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [200]:
index_list = pd.Series(dtype = 'int64')
for line in tabela_is.index.values:
    index_list = index_list.append(pd.Series(line.cdcformat(), dtype = 'int64'))
tabela_is = tabela_is.set_index(index_list)
tabela_is = tabela_is.reset_index()
tabela = tabela.merge(how = 'left', on = 'index', right = tabela_is)

In [201]:
notifica = notifica.reset_index().rename(columns={'Positividade SG':'TEMP'})
tabela = tabela.merge(how = 'left', on = tabela.index, right = notifica)
tabela = tabela[['Primeiros sintomas', 'Diagnóstico', 'Positividade SG', 'Positividade SRAG', 'SE']]
tabela = tabela.set_index('SE')

In [243]:
tabela = tabela.append(pd.DataFrame(tabela.sum(0)).transpose())

In [244]:
tabela.to_excel('output/TABELA_casos_apresentacao.xlsx', index = False)